In [1]:
import pandas as pd
import requests
import json
import glob
import os

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMC0wNy0xMVQwNzo1Mjo0Mi40NjBaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE1OTQ0NTM5NjIsImV4cCI6MTYwMTc5Nzk2MiwiYXRfaGFzaCI6InRESkRxZUJnRWpBa2N5dFNKUjdCZkEiLCJub25jZSI6IkFmTy5YSn5ZfklOaHh5M2dUZ0hyTC50QUdoRmt6ZWJjIn0.P01TNTizPXHobT3Yt8Md3gTUNkK7I9opZMilZ_iAHsU6-YHwuRQo0jjIdyqtsTvxhBzGpCsEeEez7h2C1kOnrsIF57m3O6DRnRoEeNyB52vP--z7A-zx1KrY2EuawqAVeWc5-mQCcYohSWmx75yhi0bQD3PoPbP6jNxUnmYT3O11_AbnZAM5UkB6jaAcYpsfUVXl6cB_z_R1iuWJvt5fSHJg53LPtL4waV61YEaVCbl8h9ON1Cz13-vsd_2dndxwC_CqA9gdLEw2HUF195ebtDMvVN6raYojvRZGg1EI9HVcCK0sP4q6VUspGDMZm2L97UaKCw7SYwA5zQ6WJ46bJw",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"709ec2eeb96c21bc1cfc6c573620db6646d7e838","refresh_token":"87a7c06006b3bb9c44e7d5132eac11caaeb39a1b"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,151,293957,Brodie,Grundy,COL,RUC
0,469,290528,Max,Gawn,MEL,RUC
0,755,295467,Jack,Macrae,WBD,MID
0,76,293535,Lachie,Neale,BRL,MID
0,244,291570,Nat,Fyfe,FRE,MID
...,...,...,...,...,...,...
0,818,1016214,Ayce,Taylor,ADE,DEF
0,821,1016097,Henry,Crauford,ESS,RUC
0,822,1020224,Tom,Hird,ESS,MID
0,825,297668,Nicholas,Reid,WCE,FWD


## LADDER AND FIXTURE DATA

In [5]:
# specify round
round_number = 14
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)

## COACH IDs AND NAMES

In [6]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,9806,Lester,HoweIMetYourMother
0,180650,33301,Jordan,"What's Up, Doch?"
0,45168,5524,Luke,The Dragon's Laird
0,300791,182,Paul,Coronaviney
0,21562,186,Mark,GetRichOrDieTryin
0,10857,189,Anthony,The Full Bonty
0,71052,9867,James,Bad Boys For Fyfe
0,8258,9856,Simon,It's always Sonny


In [7]:
df_coaches.to_csv('inputs/coach_list.csv')

## PLAYER MATCH RESULTS

Function to get player scores for each match

In [8]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
#     if on_field_status == True:
#         scoring_tag = 'scoring'
#     else:
#         scoring_tag = 'nonscoring'
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [9]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,86078,9806,Lester,HoweIMetYourMother,153,DEF,156,True,291313,Jeremy,Howe,COL,DEF,R1
1,2,86078,9806,Lester,HoweIMetYourMother,153,DEF,126,True,291313,Jeremy,Howe,COL,DEF,R2
2,3,86078,9806,Lester,HoweIMetYourMother,153,DEF,88,True,291313,Jeremy,Howe,COL,DEF,R3
3,4,86078,9806,Lester,HoweIMetYourMother,153,DEF,112,True,291313,Jeremy,Howe,COL,DEF,R4
4,1,86078,9806,Lester,HoweIMetYourMother,789,DEF,152,True,240283,Shannon,Hurn,WCE,DEF,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,13,300791,182,Paul,Coronaviney,359,FWD,32,True,291357,Gary,Rohan,GEE,FWD,R13
2460,13,71052,9867,James,Bad Boys For Fyfe,726,DEF,63,True,294663,Jackson,Thurlow,SYD,DEF,R13
2461,14,71052,9867,James,Bad Boys For Fyfe,726,DEF,56,True,294663,Jackson,Thurlow,SYD,DEF,R14
2462,14,180650,33301,Jordan,"What's Up, Doch?",429,DEF,88,True,294077,Michael,Hartley,HAW,DEF,R14


In [10]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [11]:
df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,86078,9806,Lester,HoweIMetYourMother,1,0,0,1827,1476,4,1
0,1,180650,33301,Jordan,"What's Up, Doch?",1,0,0,1721,1623,4,2
0,1,45168,5524,Luke,The Dragon's Laird,1,0,0,1720,1662,4,3
0,1,300791,182,Paul,Coronaviney,1,0,0,1686,1657,4,4
0,1,21562,186,Mark,GetRichOrDieTryin,0,0,1,1657,1686,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,14,8258,9856,Simon,It's always Sonny,7,0,7,23105,22678,28,4
0,14,300791,182,Paul,Coronaviney,7,0,7,22408,22709,28,5
0,14,45168,5524,Luke,The Dragon's Laird,6,0,8,23040,23338,24,6
0,14,10857,189,Anthony,The Full Bonty,4,0,10,22806,23570,16,7


In [12]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [13]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476
0,1,James vs Jordan,9867,Bad Boys For Fyfe,James,1623,33301,"What's Up, Doch?",Jordan,1721
0,2,Paul vs Anthony,182,Coronaviney,Paul,1676,189,The Full Bonty,Anthony,1653
0,2,Lester vs Mark,9806,HoweIMetYourMother,Lester,1658,186,GetRichOrDieTryin,Mark,1642
0,2,James vs Luke,9867,Bad Boys For Fyfe,James,1538,5524,The Dragon's Laird,Luke,1906
0,2,Jordan vs Simon,33301,"What's Up, Doch?",Jordan,1700,9856,It's always Sonny,Simon,1659
0,3,Paul vs Lester,182,Coronaviney,Paul,1590,9806,HoweIMetYourMother,Lester,1679
0,3,James vs Anthony,9867,Bad Boys For Fyfe,James,1263,189,The Full Bonty,Anthony,1411


In [14]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [15]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476,1.0,8.0
0,1,James vs Jordan,33301,"What's Up, Doch?",Jordan,1721,9867,Bad Boys For Fyfe,James,1623,2.0,7.0
0,1,Anthony vs Luke,5524,The Dragon's Laird,Luke,1720,189,The Full Bonty,Anthony,1662,3.0,5.0
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657,4.0,6.0
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,14,Paul vs Luke,5524,The Dragon's Laird,Luke,1631,182,Coronaviney,Paul,1660,4.0,3.0
0,14,Mark vs Simon,9856,It's always Sonny,Simon,1616,186,GetRichOrDieTryin,Mark,1796,5.0,2.0
0,14,Anthony vs Jordan,189,The Full Bonty,Anthony,1613,33301,"What's Up, Doch?",Jordan,1950,6.0,1.0
0,14,Lester vs James,9806,HoweIMetYourMother,Lester,1598,9867,Bad Boys For Fyfe,James,1593,7.0,8.0


## CURRENTLY LISTED TEAMS

In [16]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,151,owner,186.0,NaN,293957,Brodie,Grundy,COL,RUC,21562,186,Mark,GetRichOrDieTryin
1,469,owner,33301.0,NaN,290528,Max,Gawn,MEL,RUC,180650,33301,Jordan,"What's Up, Doch?"
2,755,owner,33301.0,NaN,295467,Jack,Macrae,WBD,MID,180650,33301,Jordan,"What's Up, Doch?"
3,76,owner,5524.0,NaN,293535,Lachie,Neale,BRL,MID,45168,5524,Luke,The Dragon's Laird
4,244,owner,9867.0,NaN,291570,Nat,Fyfe,FRE,MID,71052,9867,James,Bad Boys For Fyfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,147,owner,5524.0,NaN,1005054,Josh,Daicos,COL,MID,45168,5524,Luke,The Dragon's Laird
172,598,owner,189.0,NaN,1002245,Noah,Balta,RIC,RUC FWD,10857,189,Anthony,The Full Bonty
173,645,owner,33301.0,NaN,992374,Dan,Butler,STK,FWD,180650,33301,Jordan,"What's Up, Doch?"
174,190,owner,9856.0,NaN,1005577,Sam,Draper,ESS,RUC,8258,9856,Simon,It's always Sonny


## CURRENT FREE AGENTS + WAIVER

In [17]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position
32,513,free_agent,NaN,NaN,280921,Ben,Cunnington,NTH,MID
35,628,free_agent,NaN,NaN,290627,Dion,Prestia,RIC,MID
45,676,free_agent,NaN,NaN,293846,Sebastian,Ross,STK,MID
47,108,free_agent,NaN,NaN,261299,Matthew,Kreuzer,CAR,RUC
48,715,free_agent,NaN,NaN,290307,Dane,Rampe,SYD,DEF
...,...,...,...,...,...,...,...,...,...
821,818,free_agent,NaN,NaN,1016214,Ayce,Taylor,ADE,DEF
822,821,free_agent,NaN,NaN,1016097,Henry,Crauford,ESS,RUC
823,822,free_agent,NaN,NaN,1020224,Tom,Hird,ESS,MID
824,825,free_agent,NaN,NaN,297668,Nicholas,Reid,WCE,FWD


In [18]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [19]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "Waiver"
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486,291533,Tom,McDonald,MEL,Waiver
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577,293987,Jarrod,Lienert,PTA,Waiver
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802,281373,Jack,Redden,WCE,Waiver
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356,298419,Brandan,Parfitt,GEE,Waiver
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278,998129,Will,Brodie,GCS,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,15,5524,2020-09-01 04:00:02,377,293845,Jeremy,Cameron,GWS,586,993979,Sam,Powell-Pepper,PTA,Waiver
130,15,189,2020-09-01 04:00:02,598,1002245,Noah,Balta,RIC,767,291545,Mitch,Wallis,WBD,Waiver
131,15,5524,2020-09-01 04:00:02,147,1005054,Josh,Daicos,COL,277,998260,Jack,Bowes,GCS,Waiver
132,15,189,2020-09-01 04:00:02,82,290311,Mitch,Robinson,BRL,572,261892,Hamish,Hartlett,PTA,Waiver


In [20]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [21]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "Free Agent"
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev,source
0,2,182,2020-06-11 18:47:54,606,270896,Trent,Cotchin,RIC,675.0,280933,Dylan,Roberton,STK,Free Agent
1,3,182,2020-06-21 19:41:30,564,1006096,Xavier,Duursma,PTA,579.0,1004998,Todd,Marshall,PTA,Free Agent
2,3,182,2020-06-21 19:39:37,550,297354,Karl,Amon,PTA,214.0,993817,Darcy,Parish,ESS,Free Agent
3,3,182,2020-06-20 23:20:43,579,1004998,Todd,Marshall,PTA,227.0,296294,Blake,Acres,FRE,Free Agent
4,3,182,2020-06-20 15:04:59,228,295067,James,Aish,FRE,199.0,270588,Cale,Hooker,ESS,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,12,33301,2020-08-13 04:37:41,379,294508,Aidan,Corr,GWS,507.0,290787,Shaun,Atley,NTH,Free Agent
209,12,33301,2020-08-13 04:38:32,5,294307,Brad,Crouch,ADE,575.0,292145,Tom,Jonas,PTA,Free Agent
210,12,33301,2020-08-16 19:59:59,274,998130,Ben,Ainsworth,GCS,615.0,1002227,Jack,Higgins,RIC,Free Agent
211,13,33301,2020-08-19 05:55:02,507,290787,Shaun,Atley,NTH,715.0,290307,Dane,Rampe,SYD,Free Agent


In [22]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [23]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)

In [24]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'Trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

,id,trade_id,player_id,trade_player_id,round,user_team_id,processed,status,source
0,26827,16727,214,322,2,182,2020-06-12 10:01:48,2,Trade
0,30776,19019,522,574,3,186,2020-06-15 20:17:51,7,Trade
0,40668,24785,395,752,4,182,2020-07-07 00:31:22,3,Trade
0,49794,30082,90,755,5,33301,2020-06-30 21:43:08,3,Trade
1,49795,30082,570,455,5,33301,2020-06-30 21:43:08,3,Trade
2,49796,30082,389,335,5,33301,2020-06-30 21:43:08,3,Trade
3,49797,30082,491,0,5,33301,2020-06-30 21:43:08,3,Trade
4,49798,30082,637,0,5,33301,2020-06-30 21:43:08,3,Trade
0,49956,30167,36,389,5,5524,2020-07-01 19:11:42,2,Trade
0,49974,30177,36,389,5,5524,2020-06-30 22:33:56,2,Trade


In [25]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

,round,user_team_id,processed,player_id,source,player.feed_id,player.first_name,player.last_name,player.team.abbrev
0,3,186,2020-06-15 20:17:51,522,Trade,260227,Shaun,Higgins,NTH
1,3,186,2020-06-15 20:17:51,574,Trade,994295,Dan,Houston,PTA
2,5,9856,2020-07-01 21:22:52,335,Trade,270917,Patrick,Dangerfield,GEE
3,5,9856,2020-07-01 21:22:52,755,Trade,295467,Jack,Macrae,WBD
4,5,9856,2020-07-01 21:22:52,715,Trade,290307,Dane,Rampe,SYD
5,5,9856,2020-07-01 21:22:52,0,Trade,NaN,NaN,NaN,NaN
6,5,9856,2020-07-01 21:22:52,0,Trade,NaN,NaN,NaN,NaN
7,5,9856,2020-07-01 21:22:52,389,Trade,295265,Nick,Haynes,GWS
8,5,9856,2020-07-01 21:22:52,491,Trade,298210,Christian,Petracca,MEL
9,5,9856,2020-07-01 21:22:52,90,Trade,261224,Dayne,Zorko,BRL


## Combined Free Agents, Waiver, and Trades transactions list

In [26]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486.0,291533,Tom,McDonald,MEL,Waiver
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577.0,293987,Jarrod,Lienert,PTA,Waiver
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802.0,281373,Jack,Redden,WCE,Waiver
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356.0,298419,Brandan,Parfitt,GEE,Waiver
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278.0,998129,Will,Brodie,GCS,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,15,33301,2020-09-01 08:59:34,576,997142,Peter,Ladhams,PTA,NaN,NaN,NaN,NaN,NaN,Trade
99,15,189,2020-09-01 09:09:08,689,291978,Harry,Cunningham,SYD,802.0,281373,Jack,Redden,WCE,Free Agent
100,15,189,2020-09-01 09:55:21,691,998103,Oliver,Florent,SYD,82.0,290311,Mitch,Robinson,BRL,Free Agent
126,15,5524,2020-09-01 09:56:09,419,1000959,James,Cousins,HAW,319.0,290832,David,Swallow,GCS,Free Agent


In [27]:
df_transactions.to_csv('outputs/transactions.csv')

## Add source to player match results

In [28]:
def get_transaction_status(row):
    post_transaction_list = pd.DataFrame()
    round_number = row['round']
    feed_id = row['player.feed_id']
    source = row['source']
    player_rows = df_player_match_results.loc[(df_player_match_results['round'] >= round_number) & (df_player_match_results['feed_id'] == feed_id), :]
    player_rows['source'] = source
    if len(player_rows) > 0:
        df_player_match_results.update(player_rows)

df_player_match_results['source'] = "Draft"
df_transactions.apply(lambda row: get_transaction_status(row), axis=1)
df_player_match_results

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt,source
0,1.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,156.0,True,291313,Jeremy,Howe,COL,DEF,R1,Draft
1,2.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,126.0,True,291313,Jeremy,Howe,COL,DEF,R2,Draft
2,3.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,88.0,True,291313,Jeremy,Howe,COL,DEF,R3,Draft
3,4.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,112.0,True,291313,Jeremy,Howe,COL,DEF,R4,Draft
4,1.0,86078.0,9806.0,Lester,HoweIMetYourMother,789.0,DEF,152.0,True,240283,Shannon,Hurn,WCE,DEF,R1,Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2459,13.0,300791.0,182.0,Paul,Coronaviney,359.0,FWD,32.0,True,291357,Gary,Rohan,GEE,FWD,R13,Free Agent
2460,13.0,71052.0,9867.0,James,Bad Boys For Fyfe,726.0,DEF,63.0,True,294663,Jackson,Thurlow,SYD,DEF,R13,Waiver
2461,14.0,71052.0,9867.0,James,Bad Boys For Fyfe,726.0,DEF,56.0,True,294663,Jackson,Thurlow,SYD,DEF,R14,Waiver
2462,14.0,180650.0,33301.0,Jordan,"What's Up, Doch?",429.0,DEF,88.0,True,294077,Michael,Hartley,HAW,DEF,R14,Waiver


In [29]:
df_player_match_results.to_csv('outputs/player_match_results.csv')